In [2]:
from math import pi 
from ngsolve import *
from xfem import *
from netgen.occ import *
from ngsolve.webgui import *

importing ngsxfem-2.1.2504


In [3]:
def get_triangle_face(vertices):
    e1 = Segment(vertices[0],vertices[1])
    e2 = Segment(vertices[1],vertices[2])
    e3 = Segment(vertices[2],vertices[0])
    return Face(Wire([e1,e2,e3]))

In [4]:
# Define the bulk region
vertices1 = [(0,1,0),(1,0,0),(0,0,0)]
vertices2 = [(0,1,0),(1,0,0),(1,1,0)]
Omega1 = get_triangle_face(vertices1)
Omega2 = get_triangle_face(vertices2)
Omega1.faces.name = "Omega1"
Omega2.faces.name="Omega2"

In [5]:
# Define the bulk and the fracture region
background = Rectangle(1, 1).Face() 
background.edges.name="outer" # 给这个矩形的所有边命名为outer
vertices2 = [(0,1,0),(1,0,0),(1,1,0)]
Omega2 = get_triangle_face(vertices2)

Omega1 = background - Omega2 #做布尔差运算，把内矩形从外矩形中“挖掉”，形成一个有孔的区域。
Omega1.faces.name = "Omega1"
Omega2.faces.name="Omega2"

# Define names of the edges
Omega1.edges.Min(X).name = "left"
Omega1.edges.Min(Y).name = "bottom"
Omega2.edges.Max(X).name = "right"
Omega2.edges.Max(Y).name = "top"
Omega2.edges[0].name = 'fracture'

Omega2.faces.col = (1, 0, 0)

geo = Glue([Omega1, Omega2])

In [6]:
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.5))
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [7]:
print(mesh.GetBoundaries())

('left', 'fracture', 'bottom', 'right', 'top')


In [8]:
# Define the levelset function for the interface
levelset = x+y-1
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
DrawDC(lsetp1,-1,1,mesh,'lsetp1')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [9]:
# Construct the unfitted fem space 
Vh = L2(mesh,order=1,dgjumps=True)
dofs_omega1 = Vh.GetDofs(mesh.Materials("Omega1"))
dofs_omega2 = Vh.GetDofs(mesh.Materials("Omega2"))
Vh1 = Compress(Vh,dofs_omega1)
Vh2 = Compress(Vh,dofs_omega2)
ci = CutInfo(mesh,lsetp1)
Vhfbase = H1(mesh,order=1,dirichlet='top|bottom|left|right')
Vhf = Compress(Vhfbase,GetDofsOfElements(Vhfbase,ci.GetElementsOfType(IF)))
Xh = FESpace([Vh1,Vh2,Vhf])

In [96]:
print(Vh1.ndof)
print(Vh1.FreeDofs())
print(Vh2.ndof)
print(Vh2.FreeDofs())
print(Vhf.ndof)
print(Vhf.FreeDofs())

42
0: 111111111111111111111111111111111111111111
42
0: 111111111111111111111111111111111111111111
12
0: 000011111001


In [97]:
u,v = Xh.TnT()

In [98]:
normal = Normalize(grad(lsetp1))
tao = [-normal[1],normal[0]]

In [10]:
hasneg = ci.GetElementsOfType(HASNEG)
haspos = ci.GetElementsOfType(POS)
# print(hasneg)
# print(haspos)
gamma_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=haspos,bnd_val_a=False,bnd_val_b=False,use_and=True)
# print(gamma_facets)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,gamma_facets)
Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")  
dw = dFacetPatch(definedonelements=gamma_facets)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [11]:
f = CoefficientFunction(1)
u_exact = x*y

In [14]:
uf, vf = Vhf.TnT()
ds_f = ds(definedon=mesh.Boundaries("fracture"))
ds2 = dCut(levelset=lsetp1, domain_type = IF, definedonelements=ci.GetElementsOfType(IF))
a = BilinearForm(Vhf)
# a += grad(uf)*grad(vf)*dw
a += grad(uf).Trace()*grad(vf).Trace()*ds_f
# a += (grad(uf)*tao)*(grad(vf)*tao)*dw
# a += (grad(uf).Trace()*tao)*(grad(vf).Trace()*tao)*ds_f
# a += (grad(uf)*tao)*(grad(vf)*tao)*ds_f

rhs = LinearForm(Vhf)
rhs += f*vf*ds_f

a.Assemble()
rhs.Assemble()
# print(a.mat)
# print(rhs.vec) #rhs的计算应该是正确的

uh = GridFunction(Vhf)
# uh.Set(u_exact,BND)
# fh = rhs.vec.CreateVector()
# fh.data = rhs.vec - a.mat * uh.vec
# uh.vec.data += a.mat.Inverse(Vhf.FreeDofs())*fh
uh.vec.data = a.mat.Inverse(Vhf.FreeDofs())*rhs.vec
# print(uh.vec)

Row 0:   0: 2.12132   2: 0   3: -2.12132
Row 1:   1: 2.12132   4: -2.12132   5: 0
Row 2:   0: 0   2: 0   3: 0   5: 0
Row 3:   0: -2.12132   2: 0   3: 4.24264   4: -2.12132   5: 0
Row 4:   1: -2.12132   3: -2.12132   4: 4.24264   5: 0
Row 5:   1: 0   2: 0   3: 0   4: 0   5: 0



used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


In [13]:
print(rhs.vec)

 0.453047
 0.453047
       0
 0.906094
 0.906094
       0




In [102]:
# err = sqrt(Integrate((uh - u_exact)**2, mesh=mesh, definedon=mesh.Boundaries("fracture")))
err = sqrt(Integrate((uh - u_exact)**2*ds_f, mesh=mesh))
print(err)
Draw(uh,mesh)
uh2 = GridFunction(Vhf)
uh2.Set(u_exact)
# print(uh2.vec)
Draw(uh2,mesh)

0.006031070022524365


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
gfu1 = GridFunction(Vh1)
gfu1.Set(1)
Draw(gfu1,mesh)
gfu2 = GridFunction(Vh2)
gfu2.Set(-1)
Draw(gfu2,mesh)

In [ ]:
gfu = GridFunction(Vhf)
gfu.Set(1)
Draw(gfu,mesh)

In [ ]:
# Visualize the dofs
freedofs = Vh.FreeDofs()
gfu = GridFunction(Vh)
gfu.components[0].Set(1)
gfu.components[1].Set(-1)
gfu.components[2].Set(10)
DrawDC(lsetp1, gfu.components[0], gfu.components[1], mesh, "u")
Draw(gfu.components[2],mesh)
print(Vhbase.ndof, Vh.components[0].ndof, Vh.components[1].ndof,Vh.components[2].ndof)

In [ ]:
print(len(freedofs))
dofs = GetDofsOfElements(Vhbase, ci.GetElementsOfType(IF)) # 与界面相交的单元包含的自由度
print(dofs.NumSet())

In [ ]:
# Define averages and jumps

u,v = Vh.TnT()
h = specialcf.mesh_size
n = specialcf.normal(2)
jump_grad_u0 = (grad(u[0]) - grad(u[0].Other()))*n
jump_grad_v0 = (grad(v[0]) - grad(v[0].Other()))*n
jump_grad_u1 = (grad(u[1]) - grad(u[1].Other()))*n
jump_grad_v1 = (grad(v[1]) - grad(v[1].Other()))*n
jump_grad_u2 = (grad(u[2]) - grad(u[2].Other()))*n
jump_grad_v2 = (grad(v[2]) - grad(v[2].Other()))*n
jump_u = (grad(u[0]) - grad(u[1]))*n
jump_v = (grad(v[0]) - grad(v[1]))*n

In [ ]:
partialOmega1 = x*(x-1/2)*y*(1-y)
pO1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(partialOmega1,pO1)
# DrawDC(pO1,-1,1,mesh,'pO1')
c1 = CutInfo(mesh, pO1)
boundaryEle1 = c1.GetElementsOfType(IF)
# Draw(BitArrayCF(boundaryEle1), mesh, "boundaryelements")

partialOmega2 = (1-x)*(1/2-x)*y*(1-y)
pO2 = GridFunction(H1(mesh,order=1))
InterpolateToP1(partialOmega2,pO2)
c2 = CutInfo(mesh, pO2)
boundaryEle2 = c2.GetElementsOfType(IF)
# Draw(BitArrayCF(boundaryEle2), mesh, "boundaryelements")

In [ ]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
haspos = ci.GetElementsOfType(HASPOS)
hasif = ci.GetElementsOfType(IF)

# # facets used for stabilization:
fh1_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=boundaryEle1,bnd_val_a=False,bnd_val_b=False,use_and=True)
fh2_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=boundaryEle2,bnd_val_a=False,bnd_val_b=False,use_and=True)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasif, bnd_val_a=False,bnd_val_b=False,use_and=True) # all interior faces in T_{h,Gamma}

In [ ]:
# ba_surround_facets = GetElementsWithNeighborFacets(mesh,fh2_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")   
print(fh1_facets)
print(fh2_facets)
print(ba_facets)

In [ ]:
for v in mesh.vertices:
    print(v,v.point)
for el in mesh.Elements(VOL):
    # print("vertices:",el.vertices)
    # print(type(el.vertices[0]))
    print("edges:",el.edges)
    # print(type(el.edges[0]))

In [ ]:
# Construc bilinear form and right hand side 

## integral operators
dx_neg = dCut(levelset=lsetp1, domain_type = NEG, definedonelements=ci.GetElementsOfType(HASNEG))
dx_pos = dCut(levelset=lsetp1, domain_type = POS, definedonelements=ci.GetElementsOfType(HASPOS))
ds = dCut(levelset=lsetp1, domain_type = IF, definedonelements=ci.GetElementsOfType(IF))
df0 = dFacetPatch(definedonelements=fh1_facets)
df1 = dFacetPatch(definedonelements=fh2_facets)
df2 = dFacetPatch(definedonelements=ba_facets)
## Bilinear form
ah = BilinearForm(Vh,symmetric=True)
ah += grad(u[0]) * grad(v[0]) * dx_neg +  grad(u[1]) * grad(v[1]) * dx_pos + grad(u[2]) * grad(v[2]) * ds
ah += ((u[0]-u[2]) * (v[0]-v[2]) + (u[1]-u[2]) * (v[1]-v[2]) )* ds
# stabilization terms
ah += h * jump_grad_u0 * jump_grad_v0 * df0
ah += h * jump_grad_u1 * jump_grad_v1 * df1
ah += (h * jump_grad_u2 * jump_grad_v2 * df2 + h * jump_u * jump_v * ds)
ah. Assemble()

## right hand side
F = LinearForm(Vh)
F += ( f[0] * v[0] * dx_neg + f[1] * v[1] * dx_pos + f[2] * v[2] * ds)
F.Assemble()


In [ ]:
# Construc bilinear form and right hand side 

## integral operators
dx_neg = dCut(levelset=lsetp1, domain_type = NEG, definedonelements=ci.GetElementsOfType(HASNEG))
dx_pos = dCut(levelset=lsetp1, domain_type = POS, definedonelements=ci.GetElementsOfType(HASPOS))
ds = dCut(levelset=lsetp1, domain_type = IF, definedonelements=ci.GetElementsOfType(IF))
df0 = dFacetPatch(definedonelements=fh1_facets)
df1 = dFacetPatch(definedonelements=fh2_facets)
df2 = dFacetPatch(definedonelements=ba_facets)
## Bilinear form
ah = BilinearForm(Vh,symmetric=True)
ah += grad(u[0]) * grad(v[0]) * dx_neg +  grad(u[1]) * grad(v[1]) * dx_pos + grad(u[2]) * grad(v[2]) * ds
ah += ((u[0]-u[2]) * (v[0]-v[2]) + (u[1]-u[2]) * (v[1]-v[2]) )* ds
# stabilization terms
ah += h * jump_grad_u0 * jump_grad_v0 * dx(skeleton=True,definedonelements=fh1_facets)
ah += h * jump_grad_u1 * jump_grad_v1 * dx(skeleton=True,definedonelements=fh2_facets)
ah += (h * jump_grad_u2 * jump_grad_v2 * dx(skeleton=True,definedonelements=ba_facets) + h * jump_u * jump_v * dx(skeleton=True,definedonelements=ci.GetElementsOfType(IF)))
ah. Assemble()

## right hand side
F = LinearForm(Vh)
F += ( f[0] * v[0] * dx_neg + f[1] * v[1] * dx_pos + f[2] * v[2] * ds)
F.Assemble()

In [ ]:
from ngsolve import *

Vf = FacetFESpace(mesh, order=0, dirichlet=[])   # 常数向量场
n = specialcf.normal(2)
gf = GridFunction(Vf)

gf.Set(n)
Draw(gf, mesh, "n_facet")


In [ ]:
# # rl = jump_v*dx(skeleton=True,definedonelements=ci.GetElementsOfType(IF))
# rl = jump_grad_v0*df0
# ff = LinearForm(rl).Assemble()
# print(ff.vec.Norm())
ju = u[0] - u[0].Other()
jv = v[0] - v[0].Other()
a = BilinearForm(Vh)
a += (grad(u[0]) - grad(u[0].Other())) * n * (grad(v[0]) - grad(v[0].Other())) * n * df0
a.Assemble()
# print(a.mat)

for f in ba_facets:
    facet = mesh[f]
    print(f"Facet {f}, normal = {facet.Normal()}")


In [ ]:
# Solve the system

gfu = GridFunction(Vh)
freedofs = Vh.FreeDofs()
## deal with Dirichlet boundary
gfu.components[0].Set(gD,BND)
gfu.components[1].Set(gD,BND)
F.vec.data -= ah.mat * gfu.vec
gfu.vec.data += ah.mat.Inverse(freedofs) * F.vec


In [ ]:
# Calculate L2 error
# u_exact = [exact_u_neg, exact_u_pos]
err = sqrt(Integrate((gfu.components[0] - exact_u_p_neg)**2*dx_neg, mesh) + Integrate((gfu.components[1] - exact_u_p_pos)**2*dx_pos, mesh)) 
print(err)
err_f = sqrt(Integrate((gfu.components[2] - exact_u_f)**2*ds, mesh))
print(err_f)

In [ ]:
DrawDC(levelset, gfu.components[0], gfu.components[1], mesh, "u")
DrawDC(levelset, exact_u_p_neg, exact_u_p_pos, mesh)

In [ ]:
exact_u = x*(1-x)*y*(1-y)
Draw(exact_u,mesh)

gfu_orig = GridFunction(Vhbase)
gfu_orig.vec[:] = 0  # 默认设置为 0
used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(HASNEG))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] += gfu.components[0].vec[i]

used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(HASPOS))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] += gfu.components[1].vec[i]

used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(IF))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] -= gfu.components[2].vec[i]

Draw(gfu_orig,mesh)

error = sqrt(Integrate((gfu_orig - exact_u)**2*dx, mesh))
print(error)

In [ ]:
Draw(gfu.components[0], mesh, "u_neg")
Draw(gfu.components[1], mesh, "u_pos")
Draw(exact_u_p_neg, mesh, "exact_neg")
Draw(exact_u_p_pos, mesh, "exact_pos")


In [ ]:
Draw(gfu.components[2], mesh, "u_f")